# Part 1

In [9]:
example = open('example.txt', 'r').read()
puzzle = open('puzzle.txt', 'r').read()

input = puzzle

In [10]:
# Parse input

order_rules, updates = input.split('\n\n')

order_rules = order_rules.split('\n')
updates = updates.split('\n')

order_rules

# Make 2 dictionaries: one containing coming before rules and one containing coming after rules

must_come_before = {}
must_come_after = {}

for order in order_rules:
    before, after = order.split("|")
    
    # Populate must_come_before... [number]: [numbers that key must come BEFORE in an update]
    if not before in must_come_before:
        must_come_before[before] = [after]
    elif not after in must_come_before[before]:
        must_come_before[before].append(after)

    # Populate must_come_after... [number]: [numbers that key must come AFTER in an update]
    if not after in must_come_after:
        must_come_after[after] = [before]
    elif not before in must_come_after[after]:
        must_come_after[after].append(before)

#must_come_before, must_come_after

In [11]:
# Go through each update: for every number, check the numbers before it AND the numbers after it and check each against the respective dictionary

count = 0

for update in updates:
    valid = True
    numbers = update.split(',')

    # Iterate through each number in an update and get the numbers before it and the numbers after it
    previous_nums = []
    upcoming_nums = numbers.copy()

    for number in numbers:
        del upcoming_nums[0]
        #print(previous_nums, number, upcoming_nums)

        # Check numbers before
        for prev_num in previous_nums:
            if prev_num in must_come_after:
                if number in must_come_after[prev_num]:
                    #print(f'Invalid since {number} must come before {prev_num}')
                    valid = False
            
        # Check numbers after
        for next_num in previous_nums:
            if number in must_come_before:
                if next_num in must_come_before[number]:
                    #print(f'Invalid since {next_num} must come after {number}')
                    valid = False

        previous_nums.append(number)
    
    # Add up middle entries of all valid lists
    if valid:
        mid_idx = (len(numbers)-1)//2
        mid_num = int(numbers[mid_idx])
        count += mid_num

    #print('\n')

count

4905

# Part 2

In [12]:
# Start by extracting the incorrectly-ordered updates (copying part 1 code):

input = puzzle

# Parse input

order_rules, updates = input.split('\n\n')

order_rules = order_rules.split('\n')
updates = updates.split('\n')

order_rules

# Make 2 dictionaries: one containing coming before rules and one containing coming after rules

must_come_before = {}
must_come_after = {}

for order in order_rules:
    before, after = order.split("|")
    
    # Populate must_come_before... [number]: [numbers that key must come BEFORE in an update]
    if not before in must_come_before:
        must_come_before[before] = [after]
    elif not after in must_come_before[before]:
        must_come_before[before].append(after)

    # Populate must_come_after... [number]: [numbers that key must come AFTER in an update]
    if not after in must_come_after:
        must_come_after[after] = [before]
    elif not before in must_come_after[after]:
        must_come_after[after].append(before)

must_come_before, must_come_after


# Go through each update: for every number, check the numbers before it AND the numbers after it and check each against the respective dictionary

correct_order_updates = set()
incorrect_order_updates = set()

for update in updates:
    valid = True
    numbers = update.split(',')

    # Iterate through each number in an update and get the numbers before it and the numbers after it
    previous_nums = []
    upcoming_nums = numbers.copy()

    for number in numbers:
        del upcoming_nums[0]
        #print(previous_nums, number, upcoming_nums)

        # Check numbers before
        for prev_num in previous_nums:
            if prev_num in must_come_after:
                if number in must_come_after[prev_num]:
                    valid = False
            
        # Check numbers after
        for next_num in previous_nums:
            if number in must_come_before:
                if next_num in must_come_before[number]:
                    valid = False

        previous_nums.append(number)
    
    # Append invalid updates to set
    if valid:
        correct_order_updates.add(update)

    if not valid:
        incorrect_order_updates.add(update)

#incorrect_order_updates

In [13]:
def is_in_correct_order(update):
    '''
    Input: update of format ['num1', 'num2', ...]
    Output: True/False on whether update is in a valid order

    Just repurposes the part 1 code methodology into a function that returns the validity boolean
    '''
    valid = True

    # Iterate through each number in an update and get the numbers before it and the numbers after it
    previous_nums = []
    upcoming_nums = update.copy()

    for number in update:
        del upcoming_nums[0]
        #print(previous_nums, number, upcoming_nums)

        # Check numbers before
        for prev_num in previous_nums:
            if prev_num in must_come_after:
                if number in must_come_after[prev_num]:
                    valid = False
            
        # Check numbers after
        for next_num in previous_nums:
            if number in must_come_before:
                if next_num in must_come_before[number]:
                    valid = False

        previous_nums.append(number)
    
    return valid




# # Unit testing
# for u in correct_order_updates:
#     print(is_in_correct_order(u.split(',')))

# for u in incorrect_order_updates:
#     print(is_in_correct_order(u.split(',')))

In [14]:
fixed_updates = []

for update in incorrect_order_updates:
    numbers = update.split(',')
    #print(numbers)

    i = 0
    while(not is_in_correct_order(numbers)):
        previous_nums = []
        upcoming_nums = numbers.copy()
        for num_idx,number in enumerate(numbers):
            changed = False
            del upcoming_nums[0]
            #print(previous_nums, number, upcoming_nums)

            # Check numbers before
            for prev_num_idx,prev_num in enumerate(previous_nums):
                if prev_num in must_come_after:
                    if number in must_come_after[prev_num]:
                        # At invalid point, swap the numbers around
                        #print(f'Invalid since {number} must come before {prev_num}')
                        changed = True
                        new_update = numbers.copy()
                        #print(f'swapping... {new_update[num_idx], new_update[num_idx-(len(previous_nums)-prev_num_idx)]}')
                        new_update[num_idx], new_update[num_idx-(len(previous_nums)-prev_num_idx)] = new_update[num_idx-(len(previous_nums)-prev_num_idx)], new_update[num_idx]
                        #print(f'new update = {new_update}')
                        break

            if changed:
                numbers = new_update
                #print(f'trying again with {new_update}\n')
                break

            # Check numbers after
            for next_num_idx,next_num in enumerate(previous_nums):
                if number in must_come_before:
                    if next_num in must_come_before[number]:
                        # At invalid point, swap the numbers around
                        #print(f'Invalid since {next_num} must come after {number}')
                        changed = True
                        new_update = numbers.copy()
                        new_update[num_idx], new_update[num_idx+next_num_idx] = new_update[num_idx+next_num_idx], new_update[num_idx]
                        break
            
            if changed:
                numbers = new_update
                #print(f'trying again with {new_update}\n')
                break

            previous_nums.append(number)
        # Safety exit condition
        i += 1
        if i > 10000000:
            break
    
    fixed_updates.append(numbers)

In [15]:
count = 0
for fixed_update in fixed_updates:
    #print(fixed_update)
    mid_idx = len(fixed_update)//2
    count += int(fixed_update[mid_idx])
count

6204